#  Project

**Project Goal:** 

**Dataset Period:**

**Methodology:** 

---

## 1. Business Understanding

### 1.1 Business Objectives
TODO

### 1.2 Project Goals and Successs Criteria
TODO


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.model_selection import train_test_split 
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

import math
from statsmodels.nonparametric.smoothers_lowess import lowess

---
## 2. Data Understanding

### 2.1 Data Collection

The first step is to load the datasets into the working environment. This involves importing the necessary libraries and reading the data file into a suitable data structure, namely, a DataFrame using Pandas.

In [ ]:
# Load datasets
weather = pd.read_csv("weather.csv", parse_dates=["time"])
energy = pd.read_csv("energy_dataset.csv", parse_dates=["time"])

# Set 'time' as index
weather = weather.set_index("time")
energy = energy.set_index("time")

### 2.2 Data Description

##### Basic dataset information

In [ ]:
energy.info()

The *energy* dataset contains 35064 entries and 24 features, representing hourly records of electricity generation from various sources, total system load, and the day-ahead market price. Each row corresponds to one hour of energy system operation, and the goal is to forecast the electricity price one hour and one day ahead.

In [ ]:
energy.describe()

In [ ]:
energy.head()

In [ ]:
weather.info()

The *weather* dataset also contains 35064 entries and 5 features representing hourly meteorological measurements such as temperature, pressure, humidity, and wind speed. Each row corresponds to one hour of weather conditions, and these variables are used as exogenous inputs to improve electricity price forecasting.

In [ ]:
weather.describe()

In [ ]:
weather.head()

### 2.3 Data Exploration

#### 2.3.1 Target variable analysis

The target variable `price_day_ahead` represents the eletricity market price for the upcoming hour.

In [ ]:
plt.figure(figsize=(8,5))
plt.hist(energy['price_day_ahead'], bins=50, edgecolor='black')
plt.title("Histogram of Day-Ahead Electricity Price")
plt.xlabel("Price (€/MWh)")
plt.ylabel("Frequency")
plt.grid(True)
plt.show()

The histogram shows a distribution that is very close to a normal distribution, although it is slightly right-skewed.

In [ ]:
plt.figure(figsize=(14, 6))
plt.plot(energy['price_day_ahead'])
plt.title("Time Series of Electricity Price (Day-Ahead)")
plt.xlabel("Time")
plt.ylabel("Price (€/MWh)")
plt.grid(True)
plt.show()

The time series plot shows strong short-term fluctuations and clear seasonal patterns, with occasional price spikes. Prices vary over time, indicating non-stationarity and the presence of both volatility and periodic behavior.

#### 2.3.2 Feature distribution analysis

Now we will perform feature distribution analysis to examine how the data values are spread across the datasets. We will use plots and histograms to visualise the distributions features. Box plots for will be skipped in this step, as they will be specifically used for outlier detection in paragraph `2.4.2`.

In [ ]:
def feat_distribution(nonDiscreteFeatures, df):
    nrows = math.ceil(len(nonDiscreteFeatures) / 2)
    ncols = 2
    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(16, nrows * 5))
    axes = axes.flatten()

    for i, col in enumerate(nonDiscreteFeatures):
        df[col].hist(bins=30, ax=axes[i])
        axes[i].set_title(f'{col} Distribution')
        axes[i].set_xlabel(col)
        axes[i].set_ylabel('Frequency')

    plt.show()

All of the features are numerical and continuous.

##### Energy dataset

In [ ]:
energy_features = [col for col in energy.columns 
                   if col not in ['time', 'price_day_ahead']]
feat_distribution(energy_features, energy)

None of the features has a normal distribution. `generation_hydro_pumped_storage_consumption`, `generation_hydro_water_reservoir`, and `generation_solar` are strongly right-skewed, while `generation_wind_onshore` is moderately right-skewed. In contrast, `generation_waste` is left-skewed.
Variables with skewed distributions will be handled in paragraph 3.4.3, because skewness can negatively affect the models used later in paragraph 4.2, particularly KNN, SVR, and Linear Regression, which rely on distance metrics or assumptions of linearity.

`generation_marine`, `generation_geothermal`, `generation_fossil_oil_shale`, `generation_fossil_peat`, and `generation_fossil_coal-derived_gas` contain only zeros, which is consistent with the dataset description provided in paragraph 2.2.

The data represented by the remaining variables generally show irregular, multimodal, or heavily skewed shapes, reflecting the diverse and highly variable nature of electricity generation across different sources.

##### Weather dataset

In [ ]:
weather_features = [col for col in weather.columns if col != 'time']
feat_distribution(weather_features, weather)

`temperature` and `pressure` distribution is close to normal, whereas `humidity` is slightly left-skewed and `wind_speed` is right-skewed.

Skeweness will be handled in paragraph DOK

In [ ]:
skewed_cols = ['generation_hydro_pumped_storage_consumption','generation_hydro_water_reservoir','generation_solar','generation_wind_onshore','generation_waste','wind_speed']

### 2.4 Data Quality Assessment

#### 2.4.1 Identify missing values

In the first place, we will check for missing values to ensure data completeness and avoid potential issues during analysis and modeling.

In [ ]:
# % missing values by column

def missing_values_info(df):
    nulls = df.isnull().sum()
    percent = round(nulls / df.shape[0] * 100, 3)
    
    nullvalues = pd.concat([nulls, percent], axis=1)
    nullvalues.columns = ["Count", "%"]
    
    return nullvalues


In [ ]:
missing_values_info(energy)

The proportion of missing values varies across the energy-generation features. Most variables contain only a very small fraction of missing entries (around 0.05% each), including `generation_biomass`, `generation_fossil_gas`, `generation_solar`, `generation_wind_onshore`, and many others.
A few features have slightly higher but still low missing rates, such as `generation_nuclear` at 0.048% and `total_load_actual` at 0.103%.

Two variables — `generation_hydro_pumped_storage_aggregated` and `forecast_wind_offshore_eday_ahead` — have 100% missing values, meaning they contain no usable data.

In [ ]:
# skipping the NaNs
energy_features = [
    f for f in energy_features
    if energy[f].notna().sum() > 0
]

In [ ]:
missing_values_info(weather)

There are no missing values in `weather` dataset.

#### 2.4.2 Identify outliers

Based on dataframes information in paragraph `2.2` there might be potential outliers such as values at the extreme ends of the distributions (e.g., very high generation levels or unusually low/high prices). These points can disproportionately influence analysis and model results. That's why now we will identify outliers.

##### 2.4.2.1 Target variable

In [ ]:
plt.figure(figsize=(6,5))
plt.boxplot(
    energy['price_day_ahead'],
    vert=True,
    patch_artist=True,
    boxprops=dict(facecolor='lightblue', color='blue'),
    medianprops=dict(color='red'),
    whiskerprops=dict(color='blue'),
    capprops=dict(color='blue'),
    flierprops=dict(marker='o', markersize=4, markerfacecolor='blue')
)
plt.title("Boxplot of Day-Ahead Electricity Price")
plt.ylabel("Price (€/MWh)")
plt.grid(True)
plt.show()


The boxplot shows clear price spikes, confirming the presence of outliers in the target variable. These extreme values reflect real market volatility but may negatively affect several forecasting models. Therefore, while they are kept in the dataset, they will be handled later in paragraph `3.4.3` DOK through appropriate preprocessing to minimise their impact on model performance.

##### 2.4.2.2 Energy features

In [ ]:
def boxplots(numeric_features, df):
    num_plots = len(numeric_features)
    cols = 2
    rows = math.ceil(num_plots / cols)

    plt.figure(figsize=(cols * 5, rows * 4))

    for i, feature in enumerate(numeric_features):
        plt.subplot(rows, cols, i + 1)
        sns.boxplot(y=df[feature])
        plt.title(f"Boxplot: {feature}")

    plt.tight_layout()
    plt.show()

def outliers_detection(numericFeatures, df):
    Q1 = df[numericFeatures].quantile(0.25)
    Q3 = df[numericFeatures].quantile(0.75)
    IQR = Q3 - Q1

    outliers = ((df[numericFeatures] < (Q1 - 1.5 * IQR)) | (df[numericFeatures] > (Q3 + 1.5 * IQR)))
    print("Number of outliers per numeric feature:")
    print(outliers.sum())

    outliers_cols = outliers.any()
    outliers_cols = outliers_cols[outliers_cols].index.tolist()
    print("\nColumns containing outliers:")
    print(outliers_cols)


In [ ]:
boxplots(energy_features, energy)
outliers_detection(energy_features, energy)

We performed outliers detection on all numerical features using the Interquartile Range (IQR) method and visualized it using boxplots.

Features such as `generation_fossil_gas`, `generation_hydro_pumped_storage_consumption` and `generation_other` contain a high number of outlier values.
`generation_wind_onshore`, `generation_waste`,`generation_hydro_water_reservoir` and `generation_fossil_oil` also contained a noticeable number out outliers.
`generation_biomass` contains a few outliers.
The rest of the features of `energy` dataset showed no outliers according to the IQR method.

##### 2.4.2.3 Weather features

In [ ]:
boxplots(weather_features, weather)
outliers_detection(weather_features, weather)

`pressure` and `wind_speed` contain a noticeable number of outliers.

In [ ]:
outliers_cols = ['pressure', 
                 'wind_speed', 
                 'generation_fossil_gas', 
                 'generation_hydro_pumped_storage_consumption', 
                 'generation_other', 
                 'generation_wind_onshore', 
                 'generation_waste',
                 'generation_hydro_water_reservoir',
                 'generation_biomass',
                 'generation_fossil_oil' ]

#### 2.4.3 Check data duplication

In [ ]:
energy.index.duplicated().sum()


In [ ]:
weather.index.duplicated().sum()

There are no duplicates.

---
## 3. Data Preparation

### 3.1 Data Cleaning

#### 3.1.1 Missing values

The first thing to be perfomed is to handle missing values indicated in paragraph `2.4.1`.

The missing values cannot be romoved that's why the NaNs will be filled in and columns only with NaNs will be dropped.

In [ ]:
print("Remaining NaNs in energy:", energy.isna().sum().sum())

# Drop columns that contain only NaN values
energy = energy.dropna(axis=1, how='all')

# Interpolate missing values (time-series aware)
energy.index = pd.to_datetime(energy.index, errors='coerce', utc=True)
energy = energy.interpolate(method='time')

# Forward-fill remaining NaNs (edge cases)
energy = energy.ffill().bfill()

print("Remaining NaNs in energy:", energy.isna().sum().sum())


#### 3.1.2 Handling outliers

To remove outliers identified in paragraph `2.4.2` we will perform winsorization and use **Robust Scaler** during `Data Transformation` in paragraph `3.3`.


### 3.2 Bivariate analysis

Here we will skip features that consist only of values 0.

In [ ]:
df_analysis = energy.join(weather, how="inner")

# remove columns that are entirely zero
non_zero_cols = df_analysis.columns[(df_analysis != 0).any()]

# keep only numeric features
num_df = df_analysis[non_zero_cols].select_dtypes(include="number")

# compute correlation with price
corr = num_df.corr()["price_day_ahead"].drop(labels=["price_day_ahead"]).sort_values()

plt.figure(figsize=(8,12))
corr.plot(kind="barh")
plt.title("Correlation with Day-Ahead Price")
plt.xlabel("Correlation")
plt.show()


The diagram shows that some variables are correlated positively, some nagetively and some are only slightly or not correlated with the target variable at all.

#### Correlation between the eletricity price and the day of the week

In [ ]:
# print(type(energy.index))
# print(energy.index[:5])

weekday_price = energy['price_day_ahead'].groupby(energy.index.to_series().dt.dayofweek).mean()
print(weekday_price)

plt.figure(figsize=(6,4))
weekday_price.plot(kind='bar')
plt.title("Average Day-Ahead Electricity Price by Day of the Week")
plt.ylabel("Price (€/MWh)")
plt.xlabel("Day of Week (0=Mon, 6=Sun)")
plt.show()


There's no strong dependency between the day of the week and the eletricity price, however the price tends to be lower on the weekends.

##### Correlation between the temperature and the eletricity price

In [ ]:
x = df_analysis["temperature"]
y = df_analysis["price_day_ahead"]

# smooth curve
low = lowess(y, x, frac=0.15)

plt.figure(figsize=(10,6))
plt.scatter(x, y, s=5, alpha=0.1)
plt.plot(low[:,0], low[:,1], linewidth=3)

plt.title("LOWESS Smoothed Relationship: Temperature vs Price")
plt.xlabel("Temperature (K)")
plt.ylabel("Day-Ahead Price (€/MWh)")
plt.grid(True)
plt.show()


There's no significant correlation between the eletricity price and the temperature.

### 3.3 Data Transformation

Since all the features are numerical they don't have to be encoded in any way.

#### 3.3.1 Dropping unnecessary variables

There are variables with all 0 values. They will be dropped.

In [ ]:
df = energy.join(weather, how="inner")

dfML = df.copy()

zero_cols = dfML.columns[(dfML == 0).all()]
print("Dropped columns with all zeros:", list(zero_cols))

dfML = dfML.drop(columns=zero_cols)

dfML.info()


#### 3.3.1 Handling variables with skewed distribution

Since not all of the skewed distrubutions concern variables with positive values we will use **Yeo-Johnson** scaler.

In [ ]:
# Cast to float before transformation to avoid dtype issues
dfML = dfML.copy()

for col in skewed_cols:
    dfML[col] = dfML[col].astype(float)

In [ ]:
pt = PowerTransformer(method='yeo-johnson')

print(df[skewed_cols].dtypes)

dfML.loc[:, skewed_cols] = pt.fit_transform(dfML.loc[:, skewed_cols])

#### 3.3.2 Handling outliers

As indicated in paragraph `3.1`, we will use **Robust Scaler** to handle outliers.

In [ ]:
# Feature scaling

# Initialize the scaler
scaler = RobustScaler()

print(outliers_cols)
print(type(outliers_cols))

# Fit the scaler on the numeric features and transform
dfML.loc[:,outliers_cols] = scaler.fit_transform(dfML[outliers_cols])

### 3.4 Data Splitting

To evaluate the models used in paragraph `4` we have to split dataset into a train and a test set. We will use an 75/25 split ratio (the whole dataset represents 4 years of data and the test set will consist of last 12 months) to ensure sufficient amount of data for training while retaining enough samples for meaningful evaluation.

In [89]:
# Last 12 months for test
split_date = dfML.index.max() - pd.DateOffset(years=1)

train = dfML[dfML.index <= split_date]
test  = dfML[dfML.index > split_date]

print("Split date:", split_date)
print("Train:", train.shape)
print("Test:", test.shape)

Split date: 2017-12-31 22:00:00+00:00
Train: (26304, 20)
Test: (8760, 20)


### 3.5 Feature selection

#### Embedded method - Lasso regularization

In [ ]:
def lasso_regularization(df):

    X = df.iloc[:,:-1].copy()          
    y = df.iloc[:,-1].copy() 
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

    scaler = StandardScaler()
    scaler.fit(X_train)

    # fit a Logistic Regression model and feature selection altogether 
    # select the Lasso (l1) penalty.
    # The selectFromModel class from sklearn, selects the features which coefficients are non-zero

    sel_ = SelectFromModel(LogisticRegression(C=0.5, penalty='l1', solver='liblinear', random_state=10))

    sel_.fit(scaler.transform(X_train), y_train)

    # make a list with the selected features
    selected_feat = X_train.columns[(sel_.get_support())]
    
    print("Number of features which coefficient was shrank to zero: ", np.sum(sel_.estimator_.coef_ == 0))
    # identify the removed features like this:
    removed_feats = X_train.columns[(sel_.estimator_.coef_ == 0).ravel().tolist()]
    print('Removed features by Lasso: ',removed_feats) 

    return X_train.columns[(sel_.estimator_.coef_ != 0).ravel().tolist()]

In [ ]:
Lasso_SelectedColumns = lasso_regularization(train_dfML)

Lasso_SelectedColumns

print(f"Removed features by Filter methods: {FilterColumns}")

df1 = train_dfML.copy()
df1_imbalance = train_imbalance.copy()

print(df1.columns.tolist())

df1.drop(columns=FilterColumns, axis=1, inplace=True)
df1_imbalance.drop(columns=FilterColumns, axis=1, inplace=True)

df1_test = test_dfML.copy()
df1_test.drop(columns=FilterColumns, axis=1, inplace=True)

df1.info()

TODO

With these methods, we prepared a dataframes for future testing and tuning to compare different models and select the one with the best performance.

---
## 4. Modelling

### 4.1 Statistical Models

TODO

### 4.2 Machine Learning Models

#### 3.5.2 Embedded method - Lasso regularization

### 4.3 Deep Learning Models

---
## 6. Conclusions and Recommendations

### 6.1 Summary of Findings


### 6.2 Best Model Selection

### 6.3 Future Work


### 6.4 Lessons Learned


---

## Project Notes and Team Collaboration

### Team Members

- Julia Kardasz 1250264
- Mateusz Nowak 1250296
- Emilia Pawlowska 1250230